In [ ]:
#import Libraries
import json
import requests
import pprint as pp
import pandas as pd
from sqlalchemy import create_engine,inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [ ]:
#get data from API created by Manoel in C# web api and deployed at Azure
url = 'http://manoelburgos.azurewebsites.net/api/Crimes'


In [ ]:
# get content in json format
json_api_value = requests.get(url).json()

In [ ]:
#print result to know which key should be retrieved
#pp.pprint(json_api_value)

In [ ]:
# open connection with the local DB (postgres)
rds_connection_string = "postgres:filho21@localhost:5432/Crimes"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# identify the tables name
tables = engine.table_names()

In [ ]:
# identify the internal columns of the table
inspector = inspect(engine)

In [ ]:
#analyzing the 
for table in tables:
    columns = inspector.get_columns(table)
    for c in columns:
        print(f'table: {table} | column= {c["name"]} | type= {c["type"]}')

In [ ]:
# automap_base to map all the columns into object
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [ ]:
#create a table's object
City = Base.classes.city
CrimeType = Base.classes.crimestype
RegisterCrime = Base.classes.registercrime

In [ ]:
# query to avoid data duplicaton
query = session.query(City).filter(City.name=="Toronto").all()
print(query)
if query == []:
    city_one = City(name='Toronto', country="Canada", state="ON")
    session.add(city_one)
    session.commit()

In [ ]:
# getting just a single value for each type 
dic_type ={}
for value in json_api_value:
    dic_type[value['Homicide_Type'].upper()]=value['Homicide_Type'].upper()
    

In [ ]:
#printing to check results
print(dic_type)
dict_type2 = list(dic_type)
print(dict_type2)

In [ ]:
#check if the data is already there, then this avoid dupication
for data_value in dict_type2:
    query = session.query(CrimeType).filter(CrimeType.type ==data_value).all()
    if query == []:
        type_one = CrimeType(type=data_value)
        session.add(type_one)
        session.commit()

In [ ]:
#deleting content of the table, after that insert 
engine.execute("delete FROM registercrime")
session.commit()

In [ ]:
#inserting all data from json. To avoid error of integraty constraint, the ids must be selected
# from their respective main tables
for value in json_api_value:
#     find the id from the primery key to insert as foreign key
    id_city = session.query(City.id).filter(City.name==value['City']).first()
    id_crime = session.query(CrimeType.id).filter(CrimeType.type==value['Homicide_Type'].upper()).first() 
    complete_date = str(value['Occurrence_Date'])
    month =  complete_date[5:7]
    year =complete_date[0:4]
    day = complete_date[8:10]
    
    session.add(RegisterCrime(id_crime = id_crime, id_city = id_city, day=day, month=month, year=year,latitude = value['Lat'], longitude = value['Long']))
    session.commit()